In [1]:
#================================================================
# BLEU(Bilingual Evaluation Understudy) 측정 예
#
# => GPT-2와 같은 Language Model(text 생성)에서 성능 측정시 BLEU를 이용한다. 
# => BLEU는 기계번역 성능 측정 스코어로, 실제 번역 문장(references)과 모델이 생성한 문장(candidate)을 비교하여 스코어를 구한다
# => BLUE 스코어는 0~1 임, 1이면 일치한 문장, 스코어가 높을수록 성능 좋은 것임
# => NLTK에 bleu_score 라이브러리를 이용하면 됨
#
# =>자세한 내용은 https://wikidocs.net/31695 참조
#
#================================================================
from myutils import GPU_info, seed_everything, mlogging, get_bleu_scores
from tqdm.notebook import tqdm

device = GPU_info()
print(device)

#seed 설정
seed_everything(222)

#logging 설정
logger =  mlogging(loggername="gpt2-bleu", logfilename="../log/gpt2-bleu")

logfilepath:../log/bwdataset_2022-05-12.log
logfilepath:../log/qnadataset_2022-05-12.log
True
device: cuda:0
cuda index: 0
gpu 개수: 1
graphic name: NVIDIA A30
cuda:0
logfilepath:../log/gpt2-bleu_2022-05-12.log


In [2]:
# GPT-2 tokenizer, model 로딩 
import torch
from transformers import GPT2LMHeadModel, PreTrainedTokenizerFast, GPT2TokenizerFast

######################################################
model_path='../model/gpt-2/kogpt-2-ft-0504/'
######################################################

# bos_token = </s> 인 이유는 => 보통 훈련된 모델들은 </s>를 시작 과 종료 토큰으로 모두 사용한다.
tokenizer = GPT2TokenizerFast.from_pretrained(model_path,
                                                   bos_token='</s>',
                                                   eos_token='</s>',
                                                   unk_token='<unk>',
                                                   pad_token='<pad>',
                                                   mask_token='<mask>')

print(tokenizer.tokenize("<s>안녕하세요. 한국어 GPT-2 입니다."))

model = GPT2LMHeadModel.from_pretrained(model_path)
model.to(device)

The tokenizer class you load from this checkpoint is not the same type as the class this function is called from. It may result in unexpected tokenization. 
The tokenizer class you load from this checkpoint is 'PreTrainedTokenizerFast'. 
The class this function is called from is 'GPT2TokenizerFast'.


['<s>', '▁안녕', '하', '세', '요.', '▁한국어', '▁G', 'P', 'T', '-2', '▁입', '니다.']


GPT2LMHeadModel(
  (transformer): GPT2Model(
    (wte): Embedding(51200, 768)
    (wpe): Embedding(1024, 768)
    (drop): Dropout(p=0.1, inplace=False)
    (h): ModuleList(
      (0): GPT2Block(
        (ln_1): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
        (attn): GPT2Attention(
          (c_attn): Conv1D()
          (c_proj): Conv1D()
          (attn_dropout): Dropout(p=0.1, inplace=False)
          (resid_dropout): Dropout(p=0.1, inplace=False)
        )
        (ln_2): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
        (mlp): GPT2MLP(
          (c_fc): Conv1D()
          (c_proj): Conv1D()
          (dropout): Dropout(p=0.1, inplace=False)
        )
      )
      (1): GPT2Block(
        (ln_1): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
        (attn): GPT2Attention(
          (c_attn): Conv1D()
          (c_proj): Conv1D()
          (attn_dropout): Dropout(p=0.1, inplace=False)
          (resid_dropout): Dropout(p=0.1, inplace=False)
        )


In [3]:
import time
import statistics
from nltk.translate.bleu_score import sentence_bleu

#=============================================================================================
# 테스트할 말뭉치 불러옴
# => 실제 문장 뒤에 10글자는 제거한 test_sentences 문장을 만듬
#######################################################
corpus_path = "../korpora/mycorpus/2_대화체.txt"  # 말뭉치
remove_num = 5 # 뒤에서 몇개 글자를 지울지 설정값
#######################################################

org_sentences = []
test_sentences = []

with open(corpus_path, 'r', encoding='utf-8') as f:
      for line in f:
            org_sentences.append(line.strip())  # strip() 계행문자 제거
            test_sentences.append(line.strip()[:-(remove_num)])  # 뒤에 10개 문자를 제거함

#print(f'*len:{len(org_sentences)}\n')
#print(f'*org:{org_sentences[0:5]}\n')
#print(f'*test:{test_sentences[0:5]}')

start = time.time()
#=============================================================================================
# 실제 문장들을 입력하여, text generation 수행
candidates, references, scores = get_bleu_scores(model=model,                       # GPT-2 모델
                                                 tokenizer=tokenizer,               # Tokenizer
                                                 device=device,                     # cuda:0, cpu 
                                                 sentences = org_sentences[0:50],   # 입력 문장들 list
                                                 remove_token_len=remove_num,       # 뒤에서 제거할 토큰계수
                                                 show_text=True)                    # True=input, candiate, reference, blue스코어등을 printf 함

# bleu 스코어 평균을 구함    
bleu_score = statistics.mean(scores)
#=============================================================================================

# 로그 출력
logger.info(f'=== *model:{model_path}, *bleu_score:{bleu_score}, * 처리시간: {time.time() - start:.3f} 초 ===')

  0%|          | 0/50 [00:00<?, ?it/s]

*input    : 이번 신제품 출시에 대한 *blue: 0.43012508513132625 *-1
*reference: 이번 신제품 출시에 대한 시장의 반응은 어떤가요
*candidate: 이번 신제품 출시에 대한 회의가 있었습니다

*input    : 판매량이 지난번 제품보다 *blue: 5.775353993361614e-78 *-2
*reference: 판매량이 지난번 제품보다 빠르게 늘고 있습니다
*candidate: 판매량이 지난번 제품보다 많이 늘었는데  왜



/MOCOMSYS/anaconda3/envs/bong/lib/python3.9/site-packages/nltk/translate/bleu_score.py:552: UserWarning: 
The hypothesis contains 0 counts of 4-gram overlaps.
Therefore the BLEU score evaluates to 0, independently of
how many N-gram overlaps of lower order it contains.
Consider using lower n-gram order or use SmoothingFunction()
  warnings.warn(_msg)


*input    : 그렇다면 공장에 연락해서 주문량을 더 *blue: 0.44632361378533286 *-3
*reference: 그렇다면 공장에 연락해서 주문량을 더 늘려야겠네요
*candidate: 그렇다면 공장에 연락해서 주문량을 더 늘릴 수 있도록 노력

*input    : 네  제가 연락해서 주문량을 2배로 *blue: 0.44632361378533286 *-4
*reference: 네  제가 연락해서 주문량을 2배로 늘리겠습니다
*candidate: 네  제가 연락해서 주문량을 2배로 늘릴 수 있도록 하겠

*input    : 지난 회의 마지막에 논의했던 안 *blue: 0.6147881529512643 *-5
*reference: 지난 회의 마지막에 논의했던 안건을 다시 볼까요
*candidate: 지난 회의 마지막에 논의했던 안건을 마무리하고 싶은데

*input    : 그보다는 이번 주 새로운 주제가 더 *blue: 0.537284965911771 *-6
*reference: 그보다는 이번 주 새로운 주제가 더 급한 것 같습니다
*candidate: 그보다는 이번 주 새로운 주제가 더 많이 나와야 할 것

*input    : 그럼 새로운 안건으로 *blue: 5.021333847570327e-78 *-7
*reference: 그럼 새로운 안건으로 회의를 시작하도록 하죠
*candidate: 그럼 새로운 안건으로 회의 일정을 잡아야 하는데

*input    : 네  자료는 여러분의 앞에 미리 *blue: 1.0 *-8
*reference: 네  자료는 여러분의 앞에 미리 준비되어 있습니다
*candidate: 네  자료는 여러분의 앞에 미리 준비되어 있습니다

*input    : 이번 주 금요일까지 2천개를 더 *blue: 0.5169731539571706 *-9
*reference: 이번 주 금요일까지 2천개를 더 주문하라는 건가요
*candidate: 이번 주 금요일까지 2천개를 더 주문할 수 있을까

*input    : 네

2022-05-12 16:17:33,901 - gpt2-bleu - INFO - === *model:../model/gpt-2/kogpt-2-ft-0504/, *bleu_score:0.5469318501234257, * 처리시간: 7.118 초 ===


*input    : 회사 창고 중에 가장 큰 창고는 *blue: 0.6004287712485592 *-49
*reference: 회사 창고 중에 가장 큰 창고는 어느 지역의 창고인가요
*candidate: 회사 창고 중에 가장 큰 창고는 어디에 있나요

*input    : 아마 양재에 있는 창고가 *blue: 0.36409302398068727 *-50
*reference: 아마 양재에 있는 창고가 가장 크다고 알고 있어요
*candidate: 아마 양재에 있는 창고가 비어있어서  그



In [4]:
'''
# BLEU Score  예제 #1

import nltk.translate.bleu_score as bleu

# 모델에서 출력한 값
candidate = "오늘은 날씨가 흐리고 비가 옵니다."

# 실제 추정 값
references = ['내일은 날씨가 흐리고 비가 옵니다.', '오늘은 날씨가 좋고 비가 옵니다.', '오늘은 날씨가 흐리고 눈이 옵니다']

bleu_score = bleu.sentence_bleu(list(map(lambda ref: ref.split(), references)),candidate.split())
print('*bleu_score:{}'.format(bleu_score))
'''

'\n# BLEU Score  예제 #1\n\nimport nltk.translate.bleu_score as bleu\n\n# 모델에서 출력한 값\ncandidate = "오늘은 날씨가 흐리고 비가 옵니다."\n\n# 실제 추정 값\nreferences = [\'내일은 날씨가 흐리고 비가 옵니다.\', \'오늘은 날씨가 좋고 비가 옵니다.\', \'오늘은 날씨가 흐리고 눈이 옵니다\']\n\nbleu_score = bleu.sentence_bleu(list(map(lambda ref: ref.split(), references)),candidate.split())\nprint(\'*bleu_score:{}\'.format(bleu_score))\n'

In [5]:
'''
# BLEU Score  예제 #2

import statistics
from nltk.translate.bleu_score import sentence_bleu

scores = []
for i in range(len(references)):
    references_list = []
    references_list.append(references[i])  #references는 리스트로 변환해야 함
    candidate = candidates[i]
    inputtext = inputs[i]
    
    ref = list(map(lambda ref: ref.split(), references_list))
    bleu = sentence_bleu(ref, candidate.split())
    
    print(f"*input: {inputtext} *blue: {bleu}")
    print(f"*reference: {references_list[0]}")
    print(f"*candidate: {candidate}\n")
    
    scores.append(bleu)

# bleu 스코어 평균을 구함    
bleu_score = statistics.mean(scores)
'''   

'\n# BLEU Score  예제 #2\n\nimport statistics\nfrom nltk.translate.bleu_score import sentence_bleu\n\nscores = []\nfor i in range(len(references)):\n    references_list = []\n    references_list.append(references[i])  #references는 리스트로 변환해야 함\n    candidate = candidates[i]\n    inputtext = inputs[i]\n    \n    ref = list(map(lambda ref: ref.split(), references_list))\n    bleu = sentence_bleu(ref, candidate.split())\n    \n    print(f"*input: {inputtext} *blue: {bleu}")\n    print(f"*reference: {references_list[0]}")\n    print(f"*candidate: {candidate}\n")\n    \n    scores.append(bleu)\n\n# bleu 스코어 평균을 구함    \nbleu_score = statistics.mean(scores)\n'